In [1]:
import girderformindlogger
import girder_client as gc
import pandas as pd
import numpy as np

In [2]:
girder = gc.GirderClient(host="f9e93620.ngrok.io")

# Plan

- [x] create a new user (userA)
- [x] get userA's applets and make sure the list is empty
- [x] userA creates an applet with an activity-set-url (url TBD)
- [x] get userA's applets and make sure we have the applet they just created
- [x] create userB
- [ ] userA invites userB to the applet
- [ ] make sure userB has an invite
- [ ] userB accepts their invite
- [ ] list userB's applets
- [ ] userA invites userC (by email) to the applet
- [ ] create userC with the same email
- [ ] check that userC has an invite
- [ ] accept userC's applet invite
- [ ] check userC's applets
- [ ] userA, B and C submit responses to each activity in the applet
- [ ] userA can see user's B & C's data
- [ ] get responses for each user in the last 7 days
- [ ] user B removes the applet without removing data
- [ ] user C removes the applet AND removes data
- [x] delete userB and userC
- [ ] userA deactivites their applet
- [ ] delete userA


# Create a new user

In [3]:
def testCreateUser(email=None):
    randomUser = np.random.randint(1000000)
    firstName = 'test'
    lastName = 'user'
    # NOTE: girder makes login and email lowercase!!!
    login = 'testuser{}'.format(randomUser)
    if not email:
        email = 'testemail{}@testemail.com'.format(randomUser)
    password = 'password'
    createUser = girder.post('user', parameters=dict(firstName=firstName,
                                        lastName=lastName,
                                        login=login,
                                        email=email,
                                        password=password
                                       )
               )
    # assert 'authToken' in createUser.keys(), 'user has no token, {}'.format(createUser)
    assert createUser['email'] == email, 'email does not match, {} {}'.format(createUser['email'], email)
    assert createUser['firstName'] == firstName, 'firstName does not match'
    assert createUser['lastName'] == lastName, 'lastName does not match'
    assert createUser['login'] == login, 'login does not match, {} {}'.format(createUser['login'], login)
    assert createUser['admin'] == False, 'user is an admin'
    assert createUser['_modelType'] == 'user', 'model is not user'
    assert createUser['public'] == False, 'user is public!'

    return createUser

In [4]:
def authenticate(user):
    girder.authenticate(username=user['login'], password='password')

In [5]:
userA = testCreateUser()
userB = testCreateUser()

#authenticate(userA)

# Get applets when the user doesn't have any

In [6]:
def getAppletsNewUser(new_user):
    girder.authenticate(username=new_user['login'], password='password')
    appletList = girder.get('user/applets')
    assert len(appletList) == 0, 'a new user should have an empty list of applets. we get {}'.format(appletList)
    return 1

In [7]:
getAppletsNewUser(userA)

1

# get Applet by ID

want to check recently added applet here

In [8]:
def getAppletById(user, ar):
    girder.authenticate(username=user['login'], password='password')
    res = girder.get('applet/{}'.format(ar['_id']))
    assert res['applet']['_id'].split('applet/')[1] == ar['_id'], 'applet ids are not the same'
    return 1

# Add an applet

In [9]:
def addApplet(new_user):
    girder.authenticate(username=new_user['login'], password='password')

    # TODO: create an activity-set that JUST for testing.
    # make sure it has all the weird qualities that can break


    # for now, lets do the mindlogger demo
    activitySetUrl = 'http://repronim.org/schema-standardization/activity-sets/mindlogger-demo/mindlogger-demo_schema'
    randomAS = np.random.randint(1000000)
    ar = girder.post('applet', parameters=dict(activitySetUrl = activitySetUrl, name='testActivitySet{}'.format(randomAS)))

    assert ar['_id'], 'there is no ID!'
    assert ar['meta']['activitySet']['url'] == activitySetUrl, 'the URLS do not match! {} {}'.format(ar['meta']['activitySet']['url'], activitySetUrl)
    
    assert getAppletById(new_user, ar) == 1, 'something wrong with getAppletById'
    return ar

In [10]:
userA_applets = addApplet(userA)

In [11]:
def getAppletsUser(user, n=1):
    girder.authenticate(username=user['login'], password='password')
    appletList = girder.get('user/applets')
    assert len(appletList) == n, 'a new user should have {} applets. we get {}'.format(n, len(appletList))
    return 1

In [12]:
getAppletsUser(userA, 1)

1

# Invite userB to userA's applet

In [13]:
def inviteUserToApplet(userA, userA_applets, userB):
    groupId = userA_applets['roles']['user']['groups'][0]['id']
    authenticate(userA)
    inviteResp = girder.post('group/{}/invitation'.format(groupId), {
        "email": userB['email']
    })
    assert inviteResp['public'] == False, 'invite is public!'
    assert len(inviteResp['access']['users']), 'ths user was not added'

In [14]:
inviteB = inviteUserToApplet(userA, userA_applets, userB)

In [15]:
userCemail = 'randomuserc{}@test.com'.format(np.random.randint(1000000))
inviteC = inviteUserToApplet(userA, userA_applets, {'email': userCemail})

HttpError: HTTP error 500: POST https://f9e93620.ngrok.io:443/api/v1/group/5d83dcd77c925d57453cfc41/invitation?email=randomuserc444049%40test.com
Response text: {"message": "An unexpected error occurred on the server.", "type": "internal", "uid": "f6471646-471d-4b24-bee6-c14073ed526e"}

## Get a group's pending invites

In [123]:
authenticate(userA)
groupId = userA_applets['roles']['user']['groups'][0]['id']
pendingInvites = girder.get('group/{}/invitation'.format(groupId))

In [124]:
pendingInvites

[{'_accessLevel': -1,
  '_id': '5d83d8107c925d57453cfc32',
  '_modelType': 'user',
  'admin': False,
  'created': '2019-09-19T19:33:36.059000+00:00',
  'email': 'testemail136403@testemail.com',
  'firstName': 'test',
  'lastName': 'user',
  'login': 'testuser136403',
  'public': False}]

# Delete a user

In [66]:
def testDeleteUser(new_user):

    girder.authenticate(username=new_user['login'], password='password')

    deleteUser = girder.delete('user/{}'.format(new_user['_id']))

    assert deleteUser['message'] == 'Deleted user {}.'.format(new_user['login']), "{} does not equal {}".format(deleteUser['message'], 'Deleted user {}'.format(new_user['login']))
    
    return 1

In [130]:
testDeleteUser(userA)
testDeleteUser(userB)

1